# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
Arxiv has 59 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.08242


extracting tarball to tmp_2403.08242... done.
Retrieving document from  https://arxiv.org/e-print/2403.08516


extracting tarball to tmp_2403.08516...

 done.


R. Andrae  ->  R. Andrae  |  ['R. Andrae']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 69 bibliographic references in tmp_2403.08516/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2403.08698


extracting tarball to tmp_2403.08698...

 done.


Found 84 bibliographic references in tmp_2403.08698/Gaia18cjb.bbl.
syntax error in line 284: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.08516-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.08516) | **What Does the Large Magellanic Cloud Look Like? It Depends on [M/H] and  Age**  |
|| N. Frankel, et al. -- incl., <mark>R. Andrae</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-03-14*|
|*Comments*| *Submitted to ApJ; constructive comments would be appreciated*|
|**Abstract**| We offer a new way to look at the Large Magellanic Cloud through stellar mono-abundance and mono-age-mono-abundance maps. These maps are based on $\gtrsim 500\,000$ member stars with photo-spectroscopic [M/H] and age estimates from Gaia DR3 data, and they are the first area-complete, metallicity- and age-differentiated stellar maps of any disk galaxy. Azimuthally averaged, these maps reveal a surprisingly simple picture of the Milky Way's largest satellite galaxy. For any [M/H] below -0.1 dex, the LMC's radial profile is well described by a simple exponential, but with a scale length that steadily shrinks towards higher metallicities, from nearly 2.3~kpc at [M/H]$=-1.8$ to only 0.75~kpc at [M/H]$=-0.25$. The prominence of the bar decreases dramatically with [M/H], making it barely discernible at [M/H]$\lesssim -1.5$. Yet, even for metal-rich populations, the bar has little impact on the azimuthally averaged profile of the mono-abundance components. Including ages, we find that the scale length is a greater function of age than of metallicity, with younger populations far more centrally concentrated. At old ages, the scale length decreases with increasing metallicity; at young ages, the scale-length is independent of metallicity. These findings provide quantitative support for a scenario where the LMC built its stellar structure effectively outside in. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.08698-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.08698) | **The Enigma of Gaia18cjb: a Rare Hybrid of Four and EXor?**  |
|| E. Fiorellino, et al. -- incl., <mark>A. Kospal</mark> |
|*Appeared on*| *2024-03-14*|
|*Comments*| *21 pages, 13 figures*|
|**Abstract**| Context. Gaia18cjb is one of the Gaia-alerted eruptive young star candidates which has been experiencing a slow and strong brightening during the last 13 years, similar to some FU Orionis-type objects. Aims. The aim of this work is to derive the young stellar nature of Gaia18cjb, determine its physical and accretion properties to classify its variability. Methods. We conducted monitoring observations using multi-filter optical and near-infrared photometry, as well as near-infrared spectroscopy. We present the analysis of pre-outburst and outburst optical and infrared light curves, color-magnitude diagrams in different bands, the detection of near-IR spectral lines, and estimates of both stellar and accretion parameters during the burst. Results. The optical light curve shows an unusually long (8 years) brightening event of 5 mag in the last 13 years, before reaching a plateau indicating that the burst is still on-going, suggesting a FUor-like nature. The same outburst is less strong in the infrared light curves. The near-infrared spectra, obtained during the outburst, exhibit emission lines typical of highly accreting low-intermediate mass young stars with typical EXor features. The spectral index of Gaia18cjb SED classifies it as a Class I in the pre-burst stage and a Flat Spectrum young stellar object (YSO) during the burst. Conclusions. Gaia18cjb is an eruptive YSO which shows FUor-like photometric features (in terms of brightening amplitude and length of the burst) and EXor-like spectroscopic features and accretion rate, as V350 Cep and V1647 Ori, classified as objects in between FUors and EXors |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.08242-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.08242) | **Excited-State OH Masers in the Water Fountain Source IRAS 18460-0151**  |
|| X.-J. Ouyang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-03-14*|
|*Comments*| *15 pages, 3 figures, accepted for publication in ApJL*|
|**Abstract**| Water fountain objects are generally defined as "evolved stars with low to intermediate initial mass accompanied by high-velocity molecular jets detectable in the 22.235 GHz H$_2$O maser line". They are the key objects of understanding the morphological transitions of circumstellar envelopes during the post asymptotic giant branch phase. Masers are useful tools to trace the kinematic environments of the circumstellar envelopes. In this letter we report the discovery of exceptionally uncommon excited-state hydroxyl (ex-OH) masers at 4660 and 6031 MHz toward the water fountain source IRAS 18460-0151. These are the brightest ex-OH masers discovered in late-type objects to date. To the best of our knowledge, prior to the current work, no evolved stellar object has been observed in the 4660 MHz ex-OH maser line. The ground-state hydroxyl (g-OH) masers at 1612 and 1665 MHz are also observed. The velocity components of the 4660 MHz ex-OH maser line and the much weaker 1665 MHz g-OH maser line all can be seen in the 1612 MHz g-OH maser line profile. The blue-shifted components of the three masers are more intense than the red-shifted ones, in contrast to the ex-OH maser line at 6031 MHz. The relevance of the behaviors of the ex-OH masers to the circumstellar environments is unclear. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.08516.md
    + _build/html/tmp_2403.08516/./LMC-mono-abundance-maps-completeness-corrected.png
    + _build/html/tmp_2403.08516/./completeness-for-paper-a.png
    + _build/html/tmp_2403.08516/./completeness-for-paper-b.png
    + _build/html/tmp_2403.08516/./completeness-for-paper-c.png
    + _build/html/tmp_2403.08516/./completeness-for-paper-d.png
    + _build/html/tmp_2403.08516/./completeness-for-paper-full.png
exported in  _build/html/2403.08698.md
    + _build/html/tmp_2403.08698/./figures/lightcurve_tot.png
    + _build/html/tmp_2403.08698/./figures/position.png
    + _build/html/tmp_2403.08698/./figures/SED.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rbar}{\ensuremath{R_\mathrm{b}}}$
$\newcommand{\ra}{\ensuremath{\alpha}}$
$\newcommand{\dec}{\ensuremath{\delta}}$
$\newcommand{\pmra}{\ensuremath{{\mu_\alpha}\cos{\delta}}}$
$\newcommand{\pmdec}{\ensuremath{{\mu_\delta}}}$
$\newcommand{\bprp}{\ensuremath{\mathrm{BP-RP}}}$
$\newcommand{\br}{\ensuremath{\mathrm{BP-RP}}}$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{\textcolor{red}{TODO: #1}}$
$\newcommand{\Comment}[2]{ [{\color{red}\sc #1 :} {{\color{orange} \it #2}}]}$
$\newcommand{\com}[1]{ {\textcolor{cyan}{ #1}}}$
$\newcommand{\update}[1]{ {\textbf{\textcolor{mybluedark}{ #1}}}}$
$\newcommand{\updateap}[1]{ {\textbf{\textcolor{cyan}{ #1}}}}$
$\newcommand{\updatescott}[1]{ {\textbf{\textcolor{magenta}{ #1}}}}$
$\newcommand{\apn}[1]{ {\textcolor{magenta}{ #1}}}$</div>



<div id="title">

# What Does the Large Magellanic Cloud Look Like? It Depends on [M/H] and Age

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.08516-b31b1b.svg)](https://arxiv.org/abs/2403.08516)<mark>Appeared on: 2024-03-14</mark> -  _Submitted to ApJ; constructive comments would be appreciated_

</div>
<div id="authors">

N. Frankel, et al. -- incl., <mark>R. Andrae</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We offer a new way to look at the Large Magellanic Cloud through stellar mono-abundance and mono-age-mono-abundance maps. These maps are based on $\gtrsim 500 000$ member stars with photo-spectroscopic [ M/H ] and age estimates from Gaia DR3 data, and they are the first area-complete, metallicity- and age-differentiated stellar maps of any disk galaxy. Azimuthally averaged, these maps reveal a surprisingly simple picture of the Milky Way's largest satellite galaxy. For any [ M/H ] below -0.1 dex, the LMC's radial profile is well described by a simple exponential, but with a scale length that steadily shrinks towards higher metallicities, from nearly 2.3 kpc at [ M/H ] $=-1.8$ to only 0.75 kpc at [ M/H ] $=-0.25$ . The prominence of the bar decreases dramatically with [ M/H ] , making it barely discernible at [ M/H ] $\lesssim -1.5$ . Yet, even for metal-rich populations, the bar has little impact on the azimuthally averaged profile of the mono-abundance components. Including ages, we find that the scale length is a greater function of age than of metallicity, with younger populations far more centrally concentrated. At old ages, the scale length decreases with increasing metallicity; at young ages, the scale-length is independent of metallicity. These findings provide quantitative support for a scenario where the LMC built its stellar structure effectively outside in.

</div>

<div id="div_fig1">

<img src="tmp_2403.08516/./LMC-mono-abundance-maps-completeness-corrected.png" alt="Fig5" width="100%"/>

**Figure 5. -** Metallicity maps of the LMC. Panel (a) shows the global map of the mean metallicity (color coded), with total number density contours that are spaced by factors of two. Panels (b)-(f) show the completeness-corrected maps of various mono-abundance populations with \MH ranging from $-2$ to 0. All panels mask noise-dominated pixels that contain fewer than 5 stars. (*LMC-mono-abundance-maps*)

</div>
<div id="div_fig2">

<img src="tmp_2403.08516/./completeness-for-paper-a.png" alt="Fig9.1" width="25%"/><img src="tmp_2403.08516/./completeness-for-paper-b.png" alt="Fig9.2" width="25%"/><img src="tmp_2403.08516/./completeness-for-paper-c.png" alt="Fig9.3" width="25%"/><img src="tmp_2403.08516/./completeness-for-paper-d.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Completeness of the LMC according to the four factors in Eq. \ref{eq:completeness-factorised}. Panel (a) represents the last term, i.e. the probability that a star with $G < 17.5$ is in Gaia; panel (b) represents the second to last term, i.e. the probability to have an XP spectra conditioned on (a); panel (c) the second term, i.e. whether Catwise has $W_1$ and $W_2$ entries conditoinned on (a-b); and panel (d) the first term, i.e. whether a metallicity could be estimated conditioned on (a-b-c). Note the drastically different colour-bar ranges. The resulting selection function is the multiplication of those four panels and is displayed in Fig. \ref{fig:LMC-completeness}. (*fig:completeness-step-by-step*)

</div>
<div id="div_fig3">

<img src="tmp_2403.08516/./completeness-for-paper-full.png" alt="Fig1" width="100%"/>

**Figure 1. -** Probability to have \MH estimates from XGBoost for a star with apparent magnitude $G<17.65$ in the LMC footprint. This completeness map is derived in Appendix \ref{appendix:completeness} and following Eq. \ref{eq:completeness-factorised}, it is the product of the four separate maps shown in Fig. \ref{fig:completeness-step-by-step}. (*fig:LMC-completeness*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.08516"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uniF}{{\rm erg} \; {\rm s}^{-1} {\rm cm}^{-2}}$
$\newcommand{\lsun}{\mbox{L}_\odot}$
$\newcommand{\rsun}{\mbox{R}_\odot}$
$\newcommand{\msun}{\mbox{M}_\odot}$
$\newcommand{\pab}{\mbox{Pa}\beta}$
$\newcommand{\brg}{\mbox{Br}\gamma}$
$\newcommand{\lacc}{L_{\rm acc}}$
$\newcommand{\macc}{\dot{M}_{\rm acc}}$
$\newcommand{\lstar}{L_\star}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\rstar}{R_\star}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\lbol}{L_{\rm bol}}$</div>



<div id="title">

# The Enigma of Gaia18cjb: a Rare Hybrid of FUor and EXor?   \    $\thanks{Based on observations collected at the Large Binocular Telescope under LBT programme IT-2019B-008.}$   $\thanks{Based on observations collected at the European Southern Observatory under ESO/NTT programmes 105.203T.001 and 105.203T.003.}$   $\thanks{Based on observations collected at the Gran Telescopio Canarias under GTC program GTC29-22B.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.08698-b31b1b.svg)](https://arxiv.org/abs/2403.08698)<mark>Appeared on: 2024-03-14</mark> -  _21 pages, 13 figures_

</div>
<div id="authors">

E. Fiorellino, et al. -- incl., <mark>A. Kospal</mark>

</div>
<div id="abstract">

**Abstract:** Gaia18cjb is one of the Gaia-alerted eruptive young star candidates which has been experiencing a slow and strong brightening during the last 13 years, similar to some FU Orionis-type objects. The aim of this work is to derive the young stellar nature of Gaia18cjb, determine its physical and accretion properties to classify its variability. We conducted monitoring observations using multi-filter optical and near-infrared photometry, as well as near-infrared spectroscopy.We present the analysis of pre-outburst and outburst optical and infrared light curves, color-magnitude diagrams in different bands, the detection of near-IR spectral lines, and estimates of both stellar and accretion parameters during the burst. The optical light curve shows an unusually long (8 years) brightening event of 5 mag in the last 13 years, before reaching a plateau indicating that the burst is still on-going, suggesting a FUor-like nature.The same outburst is less strong in the infrared light curves.The near-infrared spectra, obtained during the outburst, exhibit emission lines typical of highly accreting low-intermediate mass young stars with typical EXor features.The spectral index of Gaia18cjb SED classifies it as a Class I in the pre-burst stage and a Flat Spectrum young stellar object (YSO) during the burst. Gaia18cjb is an eruptive YSO which shows FUor-like photometric features (in terms of brightening amplitude and length of the burst) and EXor-like spectroscopic features and accretion rate, as V350 Cep and V1647 Ori, classified as objects in between FUors and EXors.

</div>

<div id="div_fig1">

<img src="tmp_2403.08698/./figures/lightcurve_tot.png" alt="Fig7" width="100%"/>

**Figure 7. -** The visible (_ top_) and  infrared (_ bottom_) light curves of Gaia18cjb. The red circle highlights the Gaia Alert trigger. The error bars smaller than the symbol size are not presented. Grey dashed lines correspond to the epochs for which we collected NIR spectroscopy. (*fig:lightcurveVIS*)

</div>
<div id="div_fig2">

<img src="tmp_2403.08698/./figures/position.png" alt="Fig5" width="100%"/>

**Figure 5. -** The background of the figure is the Planck 857 GHz map, 10 degrees in size, centred on Gaia18cjb, the red star symbol in the plot. Blue and orange dots are sources with H$\alpha$-excess from \citet{Fratta2021} whose distances are $0.95 {\rm pc} < d < 1.05 {\rm pc}$. The orange dots are sources whose proper motions are within 3 mas/yr from Gaia18cjb proper motions, see Figure \ref{fig:magnitudes}. The regions closes to Gaia18cjb are highlighted. (*fig:gaia-location*)

</div>
<div id="div_fig3">

<img src="tmp_2403.08698/./figures/SED.png" alt="Fig3" width="100%"/>

**Figure 3. -** Spectral Energy Distribution. Circles and stars are observations taken before and after 2020, respectively. (*fig:sed*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.08698"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

388  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
